# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [136]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [137]:
import numpy as np
from random import randint
import random


np.random.seed(42)
random.seed(42)


class MySVM(object):
    def __init__(self, C=10000):
        self.C = C # regularization constant

    # f(x) = <w,x> + w_0
    def f(self, x):
        return np.dot(self.w, x) + self.w0

    # a(x) = [f(x) > 0]
    def a(self, x):
        return 1 if self.f(x) > 0 else 0
    
    # predicting answers for X_test
    def predict(self, X_test):
        return np.array([model.a(x) for x in X_test])

    # l2-regularizator
    def reg(self):
        return 1.0 * sum(self.w ** 2) / (2.0 * self.C)

    # l2-regularizator derivative
    def der_reg(self):
        ll = self.w * (1.0 / self.C)
        return ll

    # hinge loss
    def loss(self, x, answer):
        return max([0, 1 - answer * self.f(x)])

    # hinge loss derivative
    def der_loss(self, x, answer):
        hh = 1 - answer * self.f(x)
        if hh > 0:
            return -1.0
        else:
            return 0.0

    # fitting w and w_0 with SGD
    def fit(self, X_train, y_train):
        dim = len(X_train[0])
        self.w = np.random.rand(dim) # initial value for w
        self.w0 = np.random.randn() # initial value for w_0
        
        # 10000 steps is OK for this example
        # another variant is to continue iterations while error is still decreasing
        for k in range(10000):  
            
            # random example choise
            rand_index = randint(0, len(X_train) - 1) # generating random index
            x = X_train[rand_index]
            y = y_train[rand_index]

            # simple heuristic for step size
            step = 0.5 *(k+0.1)**(-0.55)

            # w update
            stepup = x * y * self.der_loss(x, y) + self.der_reg()
            self.w -= step * stepup
            
            # w_0 update
            self.w0 -= step * y * self.der_loss(x, y)

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [289]:
model = MySVM(C = 400)# C = 10000, 1000, 100, 10, 1
model.fit(X_train, y_train)
print(model.w, model.w0)

[-1.31857826 -2.84568311  1.8183376   2.7221518  -0.79159     2.95023897
 -2.26610222  0.11386951 -2.36503962 -3.66804566 -0.5345891   2.9095042
  4.38480656  2.28237382 -0.67061044  1.66643118  2.1745646   1.84944076
  0.90728417 -0.08164197] 6.322520157887064


In [290]:
predictions = model.predict(X_test)

In [291]:
print(predictions)

[1 1 1 ... 1 1 1]


In [292]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [293]:
print(len(predictions), sum(predictions))

2000 1351


In [294]:
print(sum(predictions == y_test) / float(len(y_test)))

0.747


## Задания:

### - Допишите недостающие функции в MySVM (производные и обновление весов)

### - Сравните качество с sklearn LinearSVC

In [299]:
from sklearn.svm import LinearSVC
model_LinearSVC = LinearSVC(penalty='l2', loss='hinge', max_iter=10000, C=400).fit(X_train, y_train) # C = 10000, 1000, 100, 10, 1
predictions_LinearSVC = model_LinearSVC.predict(X_test)

In [300]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ... 1 0 1] 2000 991


In [301]:
print(predictions_LinearSVC, len(predictions_LinearSVC), sum(predictions_LinearSVC))

[1 1 0 ... 1 0 1] 2000 1018


In [302]:
print(sum(predictions_LinearSVC == y_test) / float(len(y_test)))

0.7435


# Вывод:

При одинаковых параметрах accuracy(MySVM) меньше accuracy(LinearSVC).